In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from util import resolve_path

In [2]:
FILEPATH_LABS='H:/Documents/NCATS/GBM/clinicalData/Updated_All_Lab_Data_sent_to_Zhu_1-18-2023.xlsx - All_Lab_Data_Final.csv'

In [3]:
def preprocess_labs_df(filepath: str=FILEPATH_LABS) -> pd.DataFrame:
    '''Does not exit pipe.'''

    # Define the desired column types, inferring levels for categoricals
    column_types = {
        # 'Row No': 'Int64',
        # 'Data Type': 'str',
        'Subject': 'str',
        'Order_Name': 'category',
        # 'Status': 'category',
        'Collected_Datetime': 'str',  # to be parsed
        # 'btris_cluster_id': 'category',
        'btris_cluster_label': 'category',
        # 'Result_Name': 'str',
        'Result_Value_Text': 'str',
        # 'Result_Value_Numeric': 'float64',
        'Result_Value_Name': 'str',
        'Result_Note': 'str',
        'Unit_of_Measure': 'str',
        'Range': 'str',
        # 'Order_Category': 'category',
        # 'Priority': 'category',
        # 'Lab Code': 'category',
        # 'Pt Type': 'category',
        # 'Reported_Date_Time': 'str'
    }

    # Select desired columns
    selected_cols = list(column_types.keys())

    # Specify file-wide NA values
    na_values = ['', 'NULL']

    # Import data
    df = pd.read_csv(filepath, usecols=selected_cols, dtype=column_types, na_values=na_values)

    # Parse datetimes
    date_format = '%m/%d/%y %H:%M'
    df['Collected_Datetime'] = pd.to_datetime(df['Collected_Datetime'], format=date_format)
    # df['Reported_Date_Time'] = pd.to_datetime(df['Reported_Date_Time'], format=date_format)

    return df


In [4]:
def prepare_indep_lab_df(df: pd.DataFrame):
    '''Exits pipe because further processing is done elsewhere.'''

    # TK do we have to take into account whether Collected_Datetime is before Date_of_diagnosis??

    # Select desired columns
    df = df[['Subject', 'Collected_Datetime', 'btris_cluster_label', 'Result_Value_Text', 'Result_Value_Name']]

    # Fill in (at least some of) `Result_Value_Text`'s missing values with `Result_Value_Name`
    df['Result_Value_Text'] = df['Result_Value_Text'].fillna(df['Result_Value_Name'])

    # Pivot so that there's one row per `Subject`, with `Result_Value_Text` filling down each row (under their corresponding `btris_cluster_label`). The `Collected_Datetime` gets stored in a separate "level" (TK) at the (otherwise) same location as its corresponding `Result_Value_Text` measurement.
    pivoted = df.pivot(index='Subject', columns='btris_cluster_label', values=['Result_Value_Text', 'Collected_Datetime'])

    # TK what should we do with missing values? should we fill them in with random values from `Range`?

    # TK what about the textual but sort of ordinal (or even non-ordinal) data? how do we encode them ordinally?

    # Serialize the DataFrame
    pivoted.to_pickle('./results/explanatory_labs.pkl')



In [8]:
(preprocess_labs_df().pipe(prepare_indep_lab_df))

C:\Users\suns2\AppData\Local\Temp\ipykernel_11884\4052450302.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Result_Value_Text'] = df['Result_Value_Text'].fillna(df['Result_Value_Name'])


ValueError: Index contains duplicate entries, cannot reshape

In [10]:
preprocess_labs_df()

,Subject,Order_Name,Collected_Datetime,btris_cluster_label,Result_Value_Text,Result_Value_Name,Result_Note,Unit_of_Measure,Range
0,1799-Subject-00000002,CBC + Diff,2006-12-27 11:39:00,Platelet Count (Blood),157,NaN,NaN,K/uL,154-345
1,1799-Subject-00000002,CBC + Diff,2006-12-27 11:39:00,RBC (Blood),4.12,L,NaN,M/uL,4.14-5.59
2,1799-Subject-00000002,CBC + Diff,2006-12-27 11:39:00,Neutrophil % (Blood),75.4,NaN,NaN,%,40.0-78.0
3,1799-Subject-00000002,CBC + Diff,2006-12-27 11:39:00,Neutrophil Abs (Blood),3.438,NaN,NaN,K/uL,1.580-5.280
4,1799-Subject-00000002,"Creatinine, Serum",2006-12-27 11:39:00,Creatinine (Blood),1.4,NaN,NaN,mg/dL,0.9-1.4
...,...,...,...,...,...,...,...,...,...
307766,1799-Subject-00001604,PT,2021-10-22 10:37:00,PT INR (Blood),0.88,NaN,The INR is validated only as a measurement of ...,NaN,NaN
307767,1799-Subject-00001604,Thyroid Stimulating Hormone,2021-10-22 10:37:00,TSH (Blood),1.17,NaN,"Mass Spectrometry T3, T4, Reverse T3 will be p...",mcIU/mL,0.35-4.94
307768,1799-Subject-00001604,Urinalysis (includes microscopic),2021-10-22 10:37:00,Ketones (Urinalysis),TRACE,AB,NaN,NaN,NEGATIVE
307769,1799-Subject-00001604,Triiodothyronine,2021-10-22 10:37:00,T3 (Blood),77,NaN,"Mass Spectrometry T3, T4 and Reverse T3 will b...",ng/dL,58-159


In [11]:
prepare_indep_lab_df(preprocess_labs_df())

ValueError: Index contains duplicate entries, cannot reshape